In [25]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from typing import TypedDict, Annotated
from dotenv import load_dotenv
import os

load_dotenv()

True

In [26]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.95,
    top_k=40,
    google_api_key=os.getenv("GEMINI_API_KEY") 
)

In [27]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [28]:
def chat_node(state: ChatState):
    #Take the user query from the state
    messages = state['messages']

    #Send it to the LLM
    response = llm.invoke(messages)

    #response store state
    return {'messages': [response]}

In [29]:
checkpointer = MemorySaver()
graph=StateGraph(ChatState)    

# Add the node
graph.add_node('chat_node', chat_node)

#Add the edges
graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node',END)

chatbot = graph.compile(checkpointer=checkpointer)
